In [32]:
import os
import json
from pprint import pprint
from dotenv import load_dotenv
import azure.storage.blob
from azure.storage.blob import BlobServiceClient, BlobSasPermissions, generate_blob_sas,generate_container_sas,ContainerClient
from azure.storage.blob import ResourceTypes, AccountSasPermissions, generate_account_sas
import os
import sys
import datetime
from datetime import datetime, timedelta
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import azure.ai.documentintelligence
from openai import AzureOpenAI
import openpyxl 



def read_env_file(file_path):
    # Load the .env file
    load_dotenv(dotenv_path=file_path)

    # Get all environment variables
    env_vars = os.environ

    # Return the environment variables
    return env_vars

read_env_file("secrets.env")

OPENAI_ENDPOINT = os.environ.get("OPENAI_ENDPOINT")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_EMBEDDING_MODEL = os.environ.get("OPENAI_EMBEDDING_MODEL")
OPENAI_API_MODEL =  os.environ.get("OPENAI_API_MODEL")

In [33]:
#Testing 

from openai import AzureOpenAI

client = AzureOpenAI(
   azure_endpoint="https://open-ai-end-to-end-am.azure-api.net/myapi/", #do not add "/openai" at the end here because this will be automatically added by this SDK
   api_key="f9e9dbc551b14773a72b70b55700ef3e",
   api_version="2024-02-15-preview"
)

response = client.chat.completions.create(
   model="gpt-4o",
   messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"}
   ]
)
print(response)

ChatCompletion(id='chatcmpl-B7pHSMNZ3jebqa4RTSYpP0cohdfSQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Yes, Azure OpenAI Service supports customer-managed keys (CMKs). This feature allows you to use your own encryption keys to protect data associated with the service. By using CMKs, you have greater control over data encryption and can manage key rotation, revocation, and other aspects of key lifecycle management according to your policies and compliance requirements.\n\nTo use customer-managed keys with Azure OpenAI, you typically integrate with Azure Key Vault, which is a cloud service for securely storing and managing keys, secrets, and certificates. By configuring Azure OpenAI to use keys from your Key Vault, you gain enhanced control over the encryption of your data.\n\nBe sure to review the latest Azure documentation or the Azure portal for detailed instructions and any updates regarding customer-managed keys in the contex

In [34]:
def get_chat_completion(system_message, text):
    

    completion = client.chat.completions.create(
         model=OPENAI_API_MODEL,
        messages=[
         {"role": "system", "content": system_message},
        {"role": "user", "content": text}
        ]
    )

    #print(completion.choices[0].message.content)
    return completion.choices[0].message.content

In [35]:
# Install openpyxl package
#%pip install openpyxl

import openpyxl
from openpyxl import load_workbook

# Specify the path to your Excel file
file_path = "sample_data.xlsx"

# Load the workbook
workbook = load_workbook(file_path)
# Create a dictionary to hold the JSON structure
workbook_data = {}

# Iterate through all sheets in the workbook
for sheet_name in workbook.sheetnames:
    sheet = workbook[sheet_name]
    sheet_data = {}
    
    # Iterate through all rows and columns in the sheet
    for row in sheet.iter_rows():
        for cell in row:
            sheet_data[cell.coordinate] = cell.value
    
    # Add the sheet data to the workbook data
    workbook_data[sheet_name] = sheet_data

# Print the JSON structure
pprint(workbook_data)

{'Sheet1': {'A1': 'ID',
            'A2': 1,
            'A3': 3,
            'B1': 'Name',
            'B2': 'Alice',
            'B3': 'Charlie',
            'C1': 'Age',
            'C2': 25,
            'C3': 45,
            'D1': 'Status',
            'D2': 'Active',
            'D3': 'Active'}}


In [36]:
system_message = f"you are a coding assistant that uses openpyxl to write the python code it to make changes to the file data, use this file {file_path}, use the information from the file {workbook_data} once the code is generated return the source code only, remove any ``` characters"
text = "add 5 years to Charlies age" 
script = get_chat_completion(system_message, text)  
print(script)

from openpyxl import load_workbook

# Load the workbook and select the active worksheet
workbook = load_workbook('sample_data.xlsx')
sheet = workbook.active

# Add 5 years to Charlie's age
for row in sheet.iter_rows(min_row=2, min_col=1, max_col=4):
    if row[1].value == 'Charlie':
        row[2].value += 5

# Save the workbook
workbook.save('sample_data.xlsx')


In [ ]:
#Now execute the python code returned by the model

import os
import requests
import subprocess
import shutil
from pathlib import Path

script_path = Path("script.py").resolve()
with open(script_path, "w") as file:
    file.write(script)

print("Python script generated:", script_path)

python_command = shutil.which("python") or shutil.which("python3")
if not python_command:
    raise RuntimeError("Python interpreter not found.")

process = subprocess.run([python_command, str(script_path)], capture_output=True, text=True)

if process.returncode != 0:
    print("Error running Python script:", process.stderr)
else:
    print("Python script output:", process.stdout)

Python script generated: C:\Projects\OpenAIWorkshop-v2-scalable\OpenAIWorkshop-V2\script.py
Python script output: 
